In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.radarmodule as radar
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from scipy import interpolate
from metpy.plots import StationPlot
import metpy.calc as mpcalc
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
from scipy.signal import medfilt2d
import pyart
import cartopy.crs as ccrs
%matplotlib inline
import warnings;
warnings.filterwarnings('ignore')

In [ ]:
# Function definitions
def readESC(sounding_path, interpnan=True, handle=False):
    """
    Reads in a sounding in ESC format from a provided file path or handle (can be a StringIO object or an open
    file handle)
    """
    col_names = ['pressure','temperature','dewpoint','u_wind','v_wind','speed','direction','height',
                 'Qp_code','Qt_code','Qrh_code','Qu_code','Qv_code']
    # First read the file and extract the field widths from the 14th header line
    if not handle:
        f = open(sounding_path, 'r')
    else:
        f = sounding_path

    # Read in the header and extract some metadata from it
    dummy = f.readline()
    dummy = f.readline()
    header2 = f.readline().strip().split(':')
    # Read next header line and extract station id and wmo number from it (if it exists)
    staid_wmo_str = header2[1]
    if ' / ' in staid_wmo_str:
        staid_wmo = staid_wmo_str.strip().split(' / ')
        staid = staid_wmo[0][1:4]
        wmo = int(staid_wmo[1])
    else:
        if '. ' in staid_wmo_str:
            staid = staid_wmo_str.replace('. ', '').strip()[:4]
        else:
            staid = staid_wmo_str.strip()[:4]
            staid = staid.replace(" ", "")
        wmo = 99999
    print(staid)
    # Read the next header line and extract the location information from it
    header3 = f.readline().strip().split(':')
    location = header3[1].strip().split(',')
    print(location)
    lon = np.float(location[2])
    lat = np.float(location[3])
    elev = np.float(location[4])
    # Read the next header line and extract the time information from it
    header4 = f.readline().strip()[31:].lstrip()   
    sounding_datetime = datetime.strptime(header4, '%Y, %m, %d, %H:%M:%S')
    
    # Now read and dump the rest of the header
    for i in range(9):
        f.readline()
    
    # Except for the last header line, which is used to determine the widths of the fields
    line = f.readline().strip().split()
    fw = [len(field)+1 for field in line]

    # Now read the file into the dataframe, using the extracted field widths
    df = pd.read_fwf(f, usecols=[1, 2, 3, 5, 6, 7, 8, 14, 15, 16, 17, 18, 19],
                     names=col_names, na_values=['99999.0', '9999.0', '999.0'], widths=fw)
    
    # For some reason, need to convert all the columns to floating point here, as some get interpreted as strings
    # when they shouldn't be...
    # print(df['pressure'], df['temperature'])
    for column in df.columns:
        df[column] = df[column].astype(np.float)
    
    # Drop rows where height or pressure is NaN. TODO: Can't remember why I have to use reset_index(drop=True). 
    # Figure this out.
    df = df.dropna(subset=('height', 'pressure')).reset_index(drop=True)
    # Set the height as the index so we can use it as weights to interpolate other columns across NaN
    df = df.set_index('height')
    df['height'] = df.index
    
    if interpnan:
        # First convert direction and speed to u, v components
        df['u'], df['v'] = mpcalc.wind_components(df['speed'].values*units('m/s'),
                                                      df['direction'].values*units.degrees)
        # Now interpolate
        df = df.interpolate(method='values')
        # Finally recompute direction and speed from u, v components
        df['speed'] = mpcalc.wind_speed(df['u'].values*units('m/s'), df['v'].values*units('m/s'))
        df['direction'] = mpcalc.wind_direction(df['u'].values*units('m/s'), df['v'].values*units('m/s'))
    else:
        # Drop any rows with all NaN values for T, Td, winds
        df = df.dropna(subset=('temperature', 'dewpoint', 'direction', 'speed',
                               'u_wind', 'v_wind'), how='all').reset_index(drop=True)
    
    df = df[(df.Qp_code == 1.0) & (df.Qt_code == 1.0) & (df.Qrh_code == 1.0) & (df.Qu_code == 1.0) & 
            (df.Qv_code == 1.0)]

    nlines = df.count()['pressure']
    
    if not handle:
        f.close()
    
    snd_metadata = {
        'sounding_datetime': sounding_datetime,
        'lat': lat,
        'lon': lon,
        'selev': elev,
        'staid': staid,
        'wmo': wmo,
        'nlevs': nlines,
        'staid_long': staid_wmo_str
    }
    
    return snd_metadata, df


def roundPartial(value, resolution, decimals=4):
    return np.around(np.round(value / resolution) * resolution, decimals=decimals)


def rain_Brandes(D):
    """Given a range of diameters D, compute rain fall speed curve, a quartic polynomial
       fit after Brandes et al. (2002)."""
    
    D_mm=D*1000. # get it to (mm)
    
    Vtr = -0.1021 + 4.932*D_mm - 0.9551*D_mm**2. + 0.07934*D_mm**3. - 0.002362*D_mm**4.
    
    return Vtr


def cal_xf(usm,vsm,vt,H):
    """Computes final horizontal position (relative to starting position) of a raindrop
       falling through a layer H with terminal velocity vt and storm releative mean wind
       given by (usm,vsm)."""
    
    xf = (H/vt)*usm
    yf = (H/vt)*vsm
    
    return xf,yf


def mtokm(val,pos):
    """Convert m to km for formatting axes tick labels"""
    val=val/1000.0
    return '%i' % val



In [ ]:
# Read in the gridded radar data
radar_name = 'KHTX'
input_tag = 'filt_retr'
radar_type= 'NEXRAD'
date = '0430'
radar_start_timestamp = '20170430190000'
radar_end_timestamp = '20170430235959'

# Create datetime objects for start and end times
datetime_start = datetime.strptime(radar_start_timestamp, '%Y%m%d%H%M%S')
datetime_end = datetime.strptime(radar_end_timestamp, '%Y%m%d%H%M%S')

el_req = 0.5
radar_basedir = gridded_radar_dir = \
    '/Users/dawson29/sshfs_mounts/depot/data/Projects/VORTEXSE/obsdata/2017/NEXRAD/PIPS2A_FMCW/0430/HTX'
gridded_radar_dir = os.path.join(radar_basedir, 'gridded_new')
gridded_radar_paths = glob.glob(gridded_radar_dir + '/{}20170430_19*gridded*.nc'.format(radar_name))
gridded_radar_paths = gridded_radar_paths + \
    glob.glob(gridded_radar_dir + '/{}20170430_2*gridded*.nc'.format(radar_name))
gridded_radar_paths = sorted(gridded_radar_paths)
# Choose the first one for testing (2000 UTC)
gridded_radar = pyart.io.read_grid(gridded_radar_paths[0])
# print(gridded_radar.metadata)
# print(gridded_radar.projection)
# print(gridded_radar.projection_proj)

In [ ]:
# Loop through all the gridded radar files and perform masking and retrievals. Then dump updated grid files
gridded_radar_output_dir = os.path.join(radar_basedir, 'gridded_retr')
if not os.path.exists(gridded_radar_output_dir):
    os.makedirs(gridded_radar_output_dir)

# Set retrieval lookup table to use
retrieval_tag = 'Z01'
lookup_dir = os.path.join('/Users/dawson29/Projects/pyPIPS/data/lookups/', retrieval_tag)

for gridded_radar_path in gridded_radar_paths:
    gridded_radar_filename = os.path.basename(gridded_radar_path)
    print("Reading {}: ".format(gridded_radar_filename))
    gridded_radar = pyart.io.read_grid(gridded_radar_path)
    # Mask radar data where ZH < 5 dBZ and ZDR < 0.1 dB
    ZH_mask = np.where(gridded_radar.fields['reflectivity']['data'] < 5., True, False)
    ZH_mask = np.where(np.isfinite(gridded_radar.fields['reflectivity']['data']), ZH_mask, False)
    ZDR_mask = np.where(gridded_radar.fields['differential_reflectivity']['data'] < 0.1, True, False)
    ZDR_mask = np.where(np.isfinite(gridded_radar.fields['reflectivity']['data']), ZDR_mask, False)
    full_mask = np.ma.mask_or(ZH_mask, ZDR_mask)
    # full_mask = np.ma.mask_or(ZH_mask, ZDR_mask)
    for field_name in ['reflectivity', 'differential_reflectivity']:
        field_name_masked = '{}_masked'.format(field_name)
        dic = {}
        for k, v in gridded_radar.fields[field_name].items():
            if k != 'data':
                dic[k] = v
            dic['data'] = gridded_radar.fields[field_name]['data'].copy()
        gridded_radar.add_field(field_name_masked, dic, replace_existing=True)
        gridded_radar.fields[field_name_masked]['data'] = \
                np.ma.masked_array(gridded_radar.fields[field_name]['data'], mask=full_mask)
    
    # Retrieve gamma DSD parameters from ZH and ZDR
    print("Getting ZH and ZDR fields")
    # Get the ZH and ZDR fields from the radar object
    print(gridded_radar.fields.keys())
    ZH_rad_field = gridded_radar.fields['reflectivity_masked']
    ZDR_rad_field = gridded_radar.fields['differential_reflectivity_masked']
    # print(ZH_rad_tuple)
    # ZH_rad_tuple = radar.get_gridded_field_to_plot(gridded_radar, radar.REF_aliases)
    # ZDR_rad_tuple = radar.get_gridded_field_to_plot(gridded_radar, radar.ZDR_aliases)
    ZH_rad = ZH_rad_field['data']
    ZDR_rad = ZDR_rad_field['data']
    # Get the masks for both ZH_rad and ZDR_rad. These will be used later to mask the retrieved
    # values
    ZH_mask = ZH_rad.mask
    ZDR_mask = ZDR_rad.mask
    full_mask = np.ma.mask_or(ZH_mask, ZDR_mask)
    # Read in first lookup table to get the interval between reflectivity and ZDR
    lookup_path = os.path.join(lookup_dir, 'D0.csv')
    retr_table = pd.read_csv(lookup_path, sep=',', header=0, index_col='dBZ')
    # Massage the index and column labels to get rid of extraneous zeros
    # Also convert column labels from strings to floats
    retr_table.index = retr_table.index.to_series().apply(np.around, decimals=4)
    retr_table.columns = [np.around(np.float(col), decimals=4) for col in retr_table.columns]

    dBZ_lookup_min = retr_table.index[0]
    dBZ_lookup_max = retr_table.index[-1]
    ZDR_lookup_min = retr_table.columns[0]
    ZDR_lookup_max = retr_table.columns[-1]
    dBZ_intv = retr_table.index[1] - retr_table.index[0]
    ZDR_intv = float(retr_table.columns[1]) - float(retr_table.columns[0])
    # Replace masked entries in ZH_rad and ZDR_rad with the minimum value of the lookup table
    ZH_rad = ZH_rad.filled(dBZ_lookup_min)
    ZDR_rad = ZDR_rad.filled(ZDR_lookup_min)
    # Now limit values to within lookup table limits
    ZH_rad = np.where(ZH_rad < dBZ_lookup_min, dBZ_lookup_min, ZH_rad)
    ZH_rad = np.where(ZH_rad > dBZ_lookup_max, dBZ_lookup_max, ZH_rad)
    ZDR_rad = np.where(ZDR_rad < ZDR_lookup_min, ZDR_lookup_min, ZDR_rad)
    ZDR_rad = np.where(ZDR_rad > ZDR_lookup_max, ZDR_lookup_max, ZDR_rad)
    # Round ZH and ZDR fields from the radar object to the nearest interval
    ZH_round = roundPartial(ZH_rad, dBZ_intv)
    ZDR_round = roundPartial(ZDR_rad, ZDR_intv)
    # Get the shape of the arrays for later, so we can reshape the flattened arrays of retrieved
    # values
    ZH_shape = ZH_round.shape
    ZH_flat = ZH_round.flatten()
    ZDR_flat = ZDR_round.flatten()
    print(ZH_flat.max())
    print(ZDR_flat.max())
    for retr_varname in radar.retrieval_metadata.keys():
        print("Retrieving {} using lookup tables".format(retr_varname))
        lookup_path = os.path.join(lookup_dir, '{}.csv'.format(retr_varname))
        retr_table = pd.read_csv(lookup_path, sep=',', header=0, index_col='dBZ')
        # Round the indices and columns of the DataFrame (i.e. the dBZ values) to some sane
        # number of decimal places to facilitate using it as a lookup table. The floating point
        # precision gets in the way sometimes here. For example 56.4 is dumped out as
        # 56.4<some bunch of zeros>1
        retr_table.index = retr_table.index.to_series().apply(np.around, decimals=4)
        retr_table.columns = [np.around(np.float(col), decimals=4) for col in
                              retr_table.columns]
        # Gah, for some reason DataFrame.lookup sometimes barfs on perfectly good floating point
        # values in columns, so convert them back to strings here. :rolleyes:
        # EDIT 11/09/2020: Now this is happening for the rows as well. Not sure why... So change the row labels
        # to strings as well.
        retr_table.index = [str(row) for row in retr_table.index]
        retr_table.columns = [str(col) for col in retr_table.columns]
        # print(list(retr_table.index))
        # print(list(retr_table.columns))
        # Ok, now retrieve the desired retrieval variable
        # for each ZH/ZDR pair in the flattened radar sweep
    #     for ZH, ZDR in zip(ZH_flat, ZDR_flat):
    #         print(ZH, ZDR)
    #         retr_val = retr_table.lookup([ZH.astype('str')], [ZDR.astype('str')])
        retr_vals = retr_table.lookup(ZH_flat.astype('str'), ZDR_flat.astype('str'))
        # retr_vals = retr_table.lookup(ZH_flat, ZDR_flat)
        # Reshape back to original shape
        retr_vals_data = retr_vals.reshape(ZH_shape)
        retr_vals_data = np.ma.masked_array(retr_vals_data, mask=full_mask)
        # Construct the dictionary. NOTE: Gotcha! Apparently have to make a copy here,
        # otherwise contents of 'data' don't get written when same dictionary is used later.
        retr_val_dict = radar.retrieval_metadata[retr_varname].copy()
        retr_val_dict['data'] = retr_vals_data  # If I don't make a copy on the previous line
                                                        # The contents of retr_val_dict['data'] are
                                                        # not updated after the first time. This is
                                                        # very weird.
        # Save the retrieved array as a new field in the radar sweep object
        retr_varname_out = '{}_{}'.format(retr_varname, retrieval_tag)
        gridded_radar.add_field(retr_varname_out, retr_val_dict, replace_existing=True)
        # Now mask the retrieved values using the original combined mask of ZH and ZDR
        gridded_radar.fields[retr_varname_out]['data'].mask = full_mask
    
    # Dump new gridded radar object to disk
    gridded_radar_output_path = os.path.join(gridded_radar_output_dir, 'new_' + gridded_radar_filename)
    gridded_radar.write(gridded_radar_output_path, arm_time_variables=True, arm_alt_lat_lon_variables=True)

In [ ]:
# Read in PIPS data
PIPS_data_dir = '/Volumes/scr_fast/Projects/VORTEXSE/obsdata/full_PIPS_dataset_RB15/'
PIPS_filename = 'parsivel_combined_FMCW_2017_{}17_PIPS2A_60s.nc'.format(date)
PIPS_filepath = os.path.join(PIPS_data_dir, PIPS_filename)
PIPS_ds = xr.load_dataset(PIPS_filepath)
print(PIPS_ds)

In [ ]:
# Grab the location of the PIPS
PIPS_loc = eval(PIPS_ds.location)
PIPS_lat = PIPS_loc[0]
PIPS_lon = PIPS_loc[1]
print(PIPS_loc)
print(PIPS_lat, PIPS_lon)

In [ ]:
# heights of Grid levels
print(gridded_radar.metadata)
print(gridded_radar.point_z['data'][:, 10, 10])
print(gridded_radar.origin_altitude)

In [ ]:
# Plot lowest level of radar grid
projection = ccrs.PlateCarree()
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection=projection)
display = pyart.graph.GridMapDisplay(gridded_radar)
display.plot_grid('reflectivity_masked', 0, vmin=0, vmax=60, cmap='pyart_HomeyerRainbow', projection=projection)
ax.plot([PIPS_lon], [PIPS_lat], 'r*', ms=20, transform=projection)

In [ ]:
projection = ccrs.PlateCarree()
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection=projection)
display = pyart.graph.GridMapDisplay(gridded_radar)
display.plot_grid('differential_reflectivity_masked', 0, vmin=0, vmax=6, cmap='pyart_HomeyerRainbow', projection=projection)
ax.plot([PIPS_lon], [PIPS_lat], 'r*', ms=20, transform=projection)

In [ ]:
projection = ccrs.PlateCarree()
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection=projection)
display = pyart.graph.GridMapDisplay(gridded_radar)
display.plot_grid('mu_Z01', 0, vmin=0, vmax=30, cmap='pyart_HomeyerRainbow', projection=projection)
ax.plot([PIPS_lon], [PIPS_lat], 'r*', ms=20, transform=projection)

In [ ]:
projection = ccrs.PlateCarree()
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection=projection)
display = pyart.graph.GridMapDisplay(gridded_radar)
display.plot_grid('lamda_Z01', 0, vmin=0, vmax=30, cmap='pyart_HomeyerRainbow', projection=projection)
ax.plot([PIPS_lon], [PIPS_lat], 'r*', ms=20, transform=projection)

In [ ]:
projection = ccrs.PlateCarree()
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection=projection)
display = pyart.graph.GridMapDisplay(gridded_radar)
display.plot_grid('N0_Z01', 0, vmin=0, vmax=8.e3, cmap='pyart_HomeyerRainbow', projection=projection)
ax.plot([PIPS_lon], [PIPS_lat], 'r*', ms=20, transform=projection)

In [ ]:
# Plot original reflectivity at the top of the grid for comparison and sanity check

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
norm = matplotlib.colors.Normalize(vmin=0., vmax=50., clip=False)
dBZ = ax.imshow(gridded_radar_subgrid['reflectivity_masked'], origin='lower', vmin=0., vmax=50., norm=norm)
fig.colorbar(dBZ, ax=ax)